#AfriSpeech Competition

<figure>
<img src="https://raw.githubusercontent.com/owos/Stock-forecasts-dashboard/medium/Intron%20AfriSpeech-100%20Automatic%20Speech%20Recognition%20Challenge%20updated%20(1600%20%C3%97%20900%20px).png" alt="Trulli" style="width:100%">
<figcaption align = "center"><b>Figure 1:</b> Intro Afrispeech competition with Zindi

####Description
African hospitals have some of the lowest doctor-patient ratios in the world. At very busy clinics, doctors could see over 30 patients a day without any of the productivity-boosting tools available to their colleagues in developed countries.

Clinical speech-to-text is ubiquitous in the developed world but virtually absent across African hospitals. This project seeks to create pan-African English ASR models for healthcare, expanding clinical speech recognition access to African clinics to help alleviate the burden of daily clinical documentation.

The objective of this challenge is to create an automatic speech recognition model for clinical speech-to-text, using an accented English speech corpus of 200 hours, featuring 120 different African accents.
#### More on the Data
This is the largest and most diverse open-source accented speech dataset for clinical and general domain ASR in Africa, covering 13 countries, 2463 unique speakers, 52% female; featuring 67,577 audio clips, and 200 hours of audio.

In [1]:
import time
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os

# Fine-Tune Whisper For AfriSpeech Competition


Notes: The data is ~120gb and the free version  google colab can only handle 78.2GB, so we'll need to use data streaming mode to load the data.

In this Colab, we present a step-by-step guide on how to fine-tune Whisper 
for the AfriSpeech competition using Hugging Face 🤗 Transformers.

## Introduction

Whisper is a pre-trained model for automatic speech recognition (ASR) 
published in [September 2022](https://openai.com/blog/whisper/) by the authors 
Alec Radford et al. from OpenAI. Unlike many of its predecessors, such as 
[Wav2Vec 2.0](https://arxiv.org/abs/2006.11477), which are pre-trained 
on un-labelled audio data, Whisper is pre-trained on a vast quantity of 
**labelled** audio-transcription data, 680,000 hours to be precise. 
This is an order of magnitude more data than the un-labelled audio data used 
to train Wav2Vec 2.0 (60,000 hours). What is more, 117,000 hours of this 
pre-training data is multilingual ASR data. This results in checkpoints 
that can be applied to over 96 languages, many of which are considered 
_low-resource_.

When scaled to 680,000 hours of labelled pre-training data, Whisper models 
demonstrate a strong ability to generalise to many datasets and domains.
The pre-trained checkpoints achieve competitive results to state-of-the-art 
ASR systems, with near 3% word error rate (WER) on the test-clean subset of 
LibriSpeech ASR and a new state-of-the-art on TED-LIUM with 4.7% WER (_c.f._ 
Table 8 of the [Whisper paper](https://cdn.openai.com/papers/whisper.pdf)).
The extensive multilingual ASR knowledge acquired by Whisper during pre-training 
can be leveraged for other low-resource languages; through fine-tuning, the 
pre-trained checkpoints can be adapted for specific datasets and languages 
to further improve upon these results. We'll show just how Whisper can be fine-tuned 
for low-resource languages in this Colab.

<figure>
<img src="https://raw.githubusercontent.com/sanchit-gandhi/notebooks/main/whisper_architecture.svg" alt="Trulli" style="width:100%">
<figcaption align = "center"><b>Figure 1:</b> Whisper model. The architecture 
follows the standard Transformer-based encoder-decoder model. A 
log-Mel spectrogram is input to the encoder. The last encoder 
hidden states are input to the decoder via cross-attention mechanisms. The 
decoder autoregressively predicts text tokens, jointly conditional on the 
encoder hidden states and previously predicted tokens. Figure source: 
<a href="https://openai.com/blog/whisper/">OpenAI Whisper Blog</a>.</figcaption>
</figure>

The Whisper checkpoints come in five configurations of varying model sizes.
The smallest four are trained on either English-only or multilingual data.
The largest checkpoint is multilingual only. All nine of the pre-trained checkpoints 
are available on the [Hugging Face Hub](https://huggingface.co/models?search=openai/whisper). The 
checkpoints are summarised in the following table with links to the models on the Hub:

| Size   | Layers | Width | Heads | Parameters | English-only                                         | Multilingual                                      |
|--------|--------|-------|-------|------------|------------------------------------------------------|---------------------------------------------------|
| tiny   | 4      | 384   | 6     | 39 M       | [✓](https://huggingface.co/openai/whisper-tiny.en)   | [✓](https://huggingface.co/openai/whisper-tiny.)  |
| base   | 6      | 512   | 8     | 74 M       | [✓](https://huggingface.co/openai/whisper-base.en)   | [✓](https://huggingface.co/openai/whisper-base)   |
| small  | 12     | 768   | 12    | 244 M      | [✓](https://huggingface.co/openai/whisper-small.en)  | [✓](https://huggingface.co/openai/whisper-small)  |
| medium | 24     | 1024  | 16    | 769 M      | [✓](https://huggingface.co/openai/whisper-medium.en) | [✓](https://huggingface.co/openai/whisper-medium) |
| large  | 32     | 1280  | 20    | 1550 M     | x                                                    | [✓](https://huggingface.co/openai/whisper-large)  |

For demonstration purposes, we'll fine-tune the multilingual version of the 
[`"small"`](https://huggingface.co/openai/whisper-small) checkpoint with 244M params (~= 1GB). 
As for our data, we'll train and evaluate our system on a low-resource language 
taken from the [AfriSpeech](https://huggingface.co/datasets/tobiolatunji/afrispeech-200)
dataset. We'll show that with as little as 8 hours of fine-tuning data, we can achieve 
strong performance in this language.

------------------------------------------------------------------------

\\({}^1\\) The name Whisper follows from the acronym “WSPSR”, which stands for “Web-scale Supervised Pre-training for Speech Recognition”.

## Prepare Environment

First of all, let's try to secure a decent GPU for our Colab! Unfortunately, it's becoming much harder to get access to a good GPU with the free version of Google Colab. However, with Google Colab Pro one should have no issues in being allocated a V100 or P100 GPU.

To get a GPU, click _Runtime_ -> _Change runtime type_, then change _Hardware accelerator_ from _None_ to _GPU_.

We can verify that we've been assigned a GPU and view its specifications:

In [2]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

/bin/bash: /opt/conda/lib/libtinfo.so.6: no version information available (required by /bin/bash)
Thu Feb 23 18:00:58 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.82.01    Driver Version: 470.82.01    CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+--------------

Next, we need to update the Unix package `ffmpeg` to version 4:

In [3]:
!add-apt-repository -y ppa:jonathonf/ffmpeg-4
!apt update
!apt install -y ffmpeg

/bin/bash: /opt/conda/lib/libtinfo.so.6: no version information available (required by /bin/bash)
Get:1 http://packages.cloud.google.com/apt gcsfuse-focal InRelease [5002 B]
Get:2 https://packages.cloud.google.com/apt cloud-sdk InRelease [6361 B]       
Get:3 https://packages.cloud.google.com/apt google-fast-socket InRelease [5015 B]
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Hit:5 http://archive.ubuntu.com/ubuntu focal InRelease                         
Ign:6 http://ppa.launchpad.net/jonathonf/ffmpeg-4/ubuntu focal InRelease       
Get:7 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Get:8 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]        
Get:9 https://packages.cloud.google.com/apt cloud-sdk/main amd64 Packages [397 kB]
Err:10 http://ppa.launchpad.net/jonathonf/ffmpeg-4/ubuntu focal Release        
  404  Not Found [IP: 185.125.190.52 80]
Get:11 http://archive.ubuntu.com/ubuntu focal-backport

We'll employ several popular Python packages to fine-tune the Whisper model.
We'll use `datasets` to download and prepare our training data and 
`transformers` to load and train our Whisper model. We'll also require
the `soundfile` package to pre-process audio files, `evaluate` and `jiwer` to
assess the performance of our model. Finally, we'll
use `gradio` to build a flashy demo of our fine-tuned model.

In [ ]:
!pip install datasets>=2.6.1
!pip install git+https://github.com/huggingface/transformers
!pip install librosa
!pip install evaluate>=0.30
!pip install jiwer
#!pip install gradio

/bin/bash: /opt/conda/lib/libtinfo.so.6: no version information available (required by /bin/bash)
/bin/bash: /opt/conda/lib/libtinfo.so.6: no version information available (required by /bin/bash)
  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-a54gb5qh
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-a54gb5qh
  Resolved https://github.com/huggingface/transformers to commit 04d90ac49ebd4dcff31f3bb2ccf7296ee4d724b4
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 kB 2.8 MB/s eta 0:00:00a 0:00:01
  Created wheel for transformers: filename=transformers-4.27.0.dev0-py3-none-any.whl size=6624595 sha256=0b917f123fecd01a7536f5670309d93559c15d69300be93134dd08c915fe1850
  Stored in directory: /tmp/pip-ephem-wheel-cache-zs6_o_g4/wheels/35/2e/a7/d819e3310040329f

We strongly advise you to upload model checkpoints directly the [Hugging Face Hub](https://huggingface.co/) 
whilst training. The Hub provides:
- Integrated version control: you can be sure that no model checkpoint is lost during training.
- Tensorboard logs: track important metrics over the course of training.
- Model cards: document what a model does and its intended use cases.
- Community: an easy way to share and collaborate with the community!

Linking the notebook to the Hub is straightforward - it simply requires entering your 
Hub authentication token when prompted. Find your Hub authentication token [here](https://huggingface.co/settings/tokens):

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

## Load Dataset

Using 🤗 Datasets, downloading and preparing data is extremely simple. 
We can download and prepare the Afrispeech splits in just one line of code. 



In [ ]:
from datasets import load_dataset

dataset = load_dataset( "tobiolatunji/afrispeech-200", "all", use_auth_token=True, streaming=True
#     "tobiolatunji/afrispeech-200", "yoruba", use_auth_token=True, streaming=True
)

Most ASR datasets only provide input audio samples (`audio`) and the 
corresponding transcribed text (`sentence`). AfriSpeech contains additional 
metadata information, such as `accent` and `domain`, which we can disregard for ASR.
Keeping the notebook as general as possible, we only consider the input audio and
transcribed text for fine-tuning, discarding the additional metadata information:

afrispeech = `load_dataset("tobiolatunji/afrispeech-200", 'yoruba', split="validation", streaming=True)`  from the list of accents or `all`  for the entire dataset

Great! Now we can take a listen of what the audio sounds like and print the text:

In [ ]:
k = next(iter(dataset["train"]))

In [ ]:
k

In [ ]:
train_meta = pd.read_csv('../input/intron-afrispeech-csv/train_metadata.csv')
dev_meta = pd.read_csv('../input/intron-afrispeech-csv/dev_metadata.csv')
test_meta = pd.read_csv('../input/intron-afrispeech-csv/test_metadata.csv')

# test_meta = test_meta.merge(dev_meta, how='outer')
display(train_meta.head(1), os.path.join(train_meta.audio_path[0].split('/')[-2], train_meta.audio_path[0].split('/')[-1]))

In [ ]:
k['audio']['path']

In [ ]:
test_meta['test_paths'] = test_meta.audio_paths.apply(lambda i: os.path.join(i.split('/')[-2], i.split('/')[-1]))
dev_meta['test_paths'] = dev_meta.audio_paths.apply(lambda i: os.path.join(i.split('/')[-2], i.split('/')[-1]))

In [ ]:
len(dev_meta), len(test_meta)

In [ ]:
# from joblib import Parallel, delayed

# def return_path(batch):
#     path_test = batch['audio']['path']
#     if path_test in list(dev_meta['test_paths'].values):
#         return pd.DataFrame({'ID': [dev_meta[dev_meta['test_paths']==path_test]['ID'].values[0]],'transcript': [batch['transcript']]})

In [ ]:
# valid_list_paths = Parallel(n_jobs=-1, verbose=1, prefer="threads")(delayed(return_path)(batch) for batch in dataset['validation'])
# pd.concat(valid_list_paths)

In [ ]:
# train_list_paths = Parallel(n_jobs=-1, verbose=1, prefer="threads")(delayed(return_path)(batch) for batch in dataset['train'])
# pd.concat(train_list_paths)

In [ ]:
# submission = pd.concat([pd.concat(valid_list_paths), pd.concat(train_list_paths)])
# print(len(submission))

# sub = pd.read_csv('/kaggle/input/intron-afrispeech-csv/SampleSubmission.csv')
# sub['transcript'] = " "
# print(len(sub))
# sub = sub.merge(submission, on = ['ID', 'transcript'], how='outer')
# sub = sub.groupby(['ID']).tail(1).reset_index(drop=True)
# sub.to_csv('submission.csv', index=False)
# sub

In [ ]:
import IPython.display as ipd

#sample = next(iter(dataset))
audio = k["audio"]

print(k["transcript"])
ipd.Audio(data=audio["array"], autoplay=False, rate=audio["sampling_rate"])

In [ ]:
dataset_head = dataset['train'].take(2)

list(dataset_head)

In [ ]:
#sample = next(iter(dataset))
sample =list(dataset_head)[1]
audio = sample["audio"]

print(sample["transcript"])
ipd.Audio(data=audio["array"], autoplay=False, rate=audio["sampling_rate"])

### Optional: you can run the cell below and drop some of the unused columns

In [ ]:
dataset_cln = dataset.remove_columns(['speaker_id', 'path', 'age_group', 'gender', 'accent', 'domain', 'country', 'duration'])

print(dataset_cln)

In [ ]:
next(iter(dataset_cln['train']))

## Prepare Feature Extractor, Tokenizer and Data

The ASR pipeline can be de-composed into three stages: </br>
1) A feature extractor which pre-processes the raw audio-inputs </br>
2) The model which performs the sequence-to-sequence mapping </br>
3) A tokenizer which post-processes the model outputs to text format </br>

In 🤗 Transformers, the Whisper model has an associated feature extractor and tokenizer, 
called [WhisperFeatureExtractor](https://huggingface.co/docs/transformers/main/model_doc/whisper#transformers.WhisperFeatureExtractor)
and [WhisperTokenizer](https://huggingface.co/docs/transformers/main/model_doc/whisper#transformers.WhisperTokenizer) 
respectively.

We'll go through details for setting-up the feature extractor and tokenizer one-by-one!

### Load WhisperFeatureExtractor

The Whisper feature extractor performs two operations:
1. Pads / truncates the audio inputs to 30s: any audio inputs shorter than 30s are padded to 30s with silence (zeros), and those longer that 30s are truncated to 30s
2. Converts the audio inputs to _log-Mel spectrogram_ input features, a visual representation of the audio and the form of the input expected by the Whisper model

<figure>
<img src="https://raw.githubusercontent.com/sanchit-gandhi/notebooks/main/spectrogram.jpg" alt="Trulli" style="width:100%">
<figcaption align = "center"><b>Figure 2:</b> Conversion of sampled audio array to log-Mel spectrogram.
Left: sampled 1-dimensional audio signal. Right: corresponding log-Mel spectrogram. Figure source:
<a href="https://ai.googleblog.com/2019/04/specaugment-new-data-augmentation.html">Google SpecAugment Blog</a>.
</figcaption>

We'll load the feature extractor from the pre-trained checkpoint with the default values:

In [ ]:
from transformers import WhisperFeatureExtractor

feature_extractor = WhisperFeatureExtractor.from_pretrained("openai/whisper-tiny")


### Load WhisperTokenizer

The Whisper model outputs a sequence of _token ids_. The tokenizer maps each of these token ids to their corresponding text string. For English, we can load the pre-trained tokenizer and use it for fine-tuning without any further modifications. We simply have to 
specify the target language and the task. These arguments inform the 
tokenizer to prefix the language and task tokens to the start of encoded 
label sequences:

In [ ]:
from transformers import WhisperTokenizer

tokenizer = WhisperTokenizer.from_pretrained("openai/whisper-small", language="English", task="transcribe")

### Combine To Create A WhisperProcessor

To simplify using the feature extractor and tokenizer, we can _wrap_ 
both into a single `WhisperProcessor` class. This processor object 
inherits from the `WhisperFeatureExtractor` and `WhisperProcessor`, 
and can be used on the audio inputs and model predictions as required. 
In doing so, we only need to keep track of two objects during training: 
the `processor` and the `model`:

In [ ]:
from transformers import WhisperProcessor

processor = WhisperProcessor.from_pretrained("openai/whisper-small", language="English", task="transcribe")

### Prepare Data

Let's print the first example of the AfriSpeech dataset to see 
what form the data is in:

In [ ]:
print(next(iter(dataset_cln["train"])))

Since 
our input audio is sampled at 44100Hz, we need to _downsample_ it to 
16kHz prior to passing it to the Whisper feature extractor, 16kHz being the sampling rate expected by the Whisper model. 

We'll set the audio inputs to the correct sampling rate using dataset's 
[`cast_column`](https://huggingface.co/docs/datasets/package_reference/main_classes.html?highlight=cast_column#datasets.DatasetDict.cast_column)
method. This operation does not change the audio in-place, 
but rather signals to `datasets` to resample audio samples _on the fly_ the 
first time that they are loaded:

In [ ]:
# from datasets import Audio

# dataset_cln = dataset_cln["train"].cast_column("audio", Audio(sampling_rate=16000))

In [ ]:
# dataset_cln["train"].cast_column("audio", Audio(sampling_rate=16000))

def resample(batch):
    
    batch["audio"]["sampling_rate"] = 16000
    return batch

dataset_cln["train"] = dataset_cln["train"].map(resample)
dataset_cln["validation"] = dataset_cln["validation"].map(resample)

In [ ]:
next(iter(dataset_cln["train"]))

Now we can write a function to prepare our data for the model:
1. We load the resampled audio data by calling `batch["audio"]`. As explained above, 🤗 Datasets performs any necessary resampling operations on the fly.
2. We use the feature extractor to compute the log-Mel spectrogram input features from our 1-dimensional audio array.
3. We encode the transcriptions to label ids through the use of the tokenizer.

In [ ]:
def prepare_dataset(batch):
    audio = batch["audio"]

    # compute log-Mel input features from input audio array 
    batch["input_features"] = feature_extractor(audio["array"], sampling_rate=audio["sampling_rate"]).input_features[0]

    # encode target text to label ids 
    batch["labels"] = tokenizer(batch["transcript"]).input_ids
    return batch

We can apply the data preparation function to all of our training examples using dataset's `.map` method.

In [ ]:
dataset_cln = dataset_cln.map(prepare_dataset)

## Training and Evaluation

Now that we've prepared our data, we're ready to dive into the training pipeline. 
The [🤗 Trainer](https://huggingface.co/transformers/master/main_classes/trainer.html?highlight=trainer)
will do much of the heavy lifting for us. All we have to do is:

- Define a data collator: the data collator takes our pre-processed data and prepares PyTorch tensors ready for the model.

- Evaluation metrics: during evaluation, we want to evaluate the model using the [word error rate (WER)](https://huggingface.co/metrics/wer) metric. We need to define a `compute_metrics` function that handles this computation.

- Load a pre-trained checkpoint: we need to load a pre-trained checkpoint and configure it correctly for training.

- Define the training configuration: this will be used by the 🤗 Trainer to define the training schedule.

Once we've fine-tuned the model, we will evaluate it on the test data to verify that we have correctly trained it 
to transcribe speech in English.

### Define a Data Collator

The data collator for a sequence-to-sequence speech model is unique in the sense that it 
treats the `input_features` and `labels` independently: the  `input_features` must be 
handled by the feature extractor and the `labels` by the tokenizer.

The `input_features` are already padded to 30s and converted to a log-Mel spectrogram 
of fixed dimension by action of the feature extractor, so all we have to do is convert the `input_features`
to batched PyTorch tensors. We do this using the feature extractor's `.pad` method with `return_tensors=pt`.

The `labels` on the other hand are un-padded. We first pad the sequences
to the maximum length in the batch using the tokenizer's `.pad` method. The padding tokens 
are then replaced by `-100` so that these tokens are **not** taken into account when 
computing the loss. We then cut the BOS token from the start of the label sequence as we 
append it later during training.

We can leverage the `WhisperProcessor` we defined earlier to perform both the 
feature extractor and the tokenizer operations:

In [ ]:
import torch

from dataclasses import dataclass
from typing import Any, Dict, List, Union

@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lengths and need different padding methods
        # first treat the audio inputs by simply returning torch tensors
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

        # get the tokenized label sequences
        label_features = [{"input_ids": feature["labels"]} for feature in features]
        # pad the labels to max length
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        # if bos token is appended in previous tokenization step,
        # cut bos token here as it's append later anyways
        if (labels[:, 0] == self.processor.tokenizer.bos_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels

        return batch

Let's initialise the data collator we've just defined:

In [ ]:
data_collator = DataCollatorSpeechSeq2SeqWithPadding(processor=processor)

### Evaluation Metrics

We'll use the word error rate (WER) metric, the 'de-facto' metric for assessing 
ASR systems. For more information, refer to the WER [docs](https://huggingface.co/metrics/wer). We'll load the WER metric from 🤗 Evaluate:

In [ ]:
import evaluate

metric = evaluate.load("wer")

We then simply have to define a function that takes our model 
predictions and returns the WER metric. This function, called
`compute_metrics`, first replaces `-100` with the `pad_token_id`
in the `label_ids` (undoing the step we applied in the 
data collator to ignore padded tokens correctly in the loss).
It then decodes the predicted and label ids to strings. Finally,
it computes the WER between the predictions and reference labels:

In [ ]:
def compute_metrics(pred):
    pred_ids = pred.predictions
    label_ids = pred.label_ids

    # replace -100 with the pad_token_id
    label_ids[label_ids == -100] = tokenizer.pad_token_id

    # we do not want to group tokens when computing the metrics
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = tokenizer.batch_decode(label_ids, skip_special_tokens=True)

    wer = 100 * metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

### Load a Pre-Trained Checkpoint

Now let's load the pre-trained Whisper `small` checkpoint. Again, this 
is trivial through use of 🤗 Transformers!

In [ ]:
from transformers import WhisperForConditionalGeneration

model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-small")

Override generation arguments - no tokens are forced as decoder outputs (see [`forced_decoder_ids`](https://huggingface.co/docs/transformers/main_classes/text_generation#transformers.generation_utils.GenerationMixin.generate.forced_decoder_ids)), no tokens are suppressed during generation (see [`suppress_tokens`](https://huggingface.co/docs/transformers/main_classes/text_generation#transformers.generation_utils.GenerationMixin.generate.suppress_tokens)):

In [ ]:
model.config.forced_decoder_ids = None
model.config.suppress_tokens = []
model.config.use_cache = False

### Define the Training Configuration

In the final step, we define all the parameters related to training. For more detail on the training arguments, refer to the Seq2SeqTrainingArguments [docs](https://huggingface.co/docs/transformers/main_classes/trainer#transformers.Seq2SeqTrainingArguments).

In [ ]:
"""from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir="./dsn_afrispeech",  # change to a repo name of your choice dsn_afrispeech
    per_device_train_batch_size=8,
    gradient_accumulation_steps=1,  # increase by 2x for every 2x decrease in batch size
    learning_rate=1e-5,
    warmup_steps=500,
    max_steps=4000,
    num_train_epochs=1,
    gradient_checkpointing=True,
    fp16=torch.cuda.is_available(),
    evaluation_strategy="steps",
    per_device_eval_batch_size=8,  #try 4 and see if it crashes
    predict_with_generate=True,
    generation_max_length=225,
    save_steps=500,
    eval_steps=500,
    logging_steps=25,
    report_to=["wandb"],
    load_best_model_at_end=True,
    metric_for_best_model="wer",
    greater_is_better=False,
    #push_to_hub=True,
)"""

from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir="./dsn_afrispeech",  # change to a repo name of your choice dsn_afrispeech
    per_device_train_batch_size=8,
    gradient_accumulation_steps=1,  # increase by 2x for every 2x decrease in batch size
    learning_rate=1e-5,
    warmup_steps=1, #500,
    max_steps=1, #4000,
    gradient_checkpointing=True,
    fp16=torch.cuda.is_available(),
    evaluation_strategy="steps",
    per_device_eval_batch_size=8, #try 4 and see if it crashes
    predict_with_generate=True,
    generation_max_length=225,
    save_steps=1, #500,
    eval_steps=1, #500,
    logging_steps=25,
    report_to=["wandb"],
    load_best_model_at_end=True,
    metric_for_best_model="wer",
    greater_is_better=False,
    #push_to_hub=True,
)

**Note**: if one does not want to upload the model checkpoints to the Hub, 
set `push_to_hub=False`.

We can forward the training arguments to the 🤗 Trainer along with our model,
dataset, data collator and `compute_metrics` function:

In [ ]:
from transformers import Seq2SeqTrainer

train_dataset = dataset_cln['train'].skip(300).with_format("torch")
eval_dataset = dataset_cln['train'].take(300).with_format("torch")

trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=train_dataset,  
    eval_dataset=eval_dataset,  
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=processor.feature_extractor,
)

We'll save the processor object once before starting training. Since the processor is not trainable, it won't change over the course of training:

In [ ]:
processor.save_pretrained(training_args.output_dir)

### Training

Training will take a number of hours depending on your GPU or the one 
allocated to this Google Colab. If using this Google Colab directly to 
fine-tune a Whisper model, you should make sure that training isn't 
interrupted due to inactivity. A simple workaround to prevent this is 
to paste the following code into the console of this tab (_right mouse click_ 
-> _inspect_ -> _Console tab_ -> _insert code_).

```javascript
function ConnectButton(){
    console.log("Connect pushed"); 
    document.querySelector("#top-toolbar > colab-connect-button").shadowRoot.querySelector("#connect").click() 
}
setInterval(ConnectButton, 60000);
```

The peak GPU memory for the given training configuration is approximately 15.8GB. 
Depending on the GPU allocated to the Google Colab, it is possible that you will encounter a CUDA `"out-of-memory"` error when you launch training. 
In this case, you can reduce the `per_device_train_batch_size` incrementally by factors of 2 
and employ [`gradient_accumulation_steps`](https://huggingface.co/docs/transformers/main_classes/trainer#transformers.Seq2SeqTrainingArguments.gradient_accumulation_steps)
to compensate.

To launch training, simply execute:

In [ ]:
#this empties the GPU
torch.cuda.empty_cache()

In [ ]:
trainer.train()

###saving the Model to be loader to be used later in the future.

In [ ]:
trainer.save_model(training_args.output_dir)

In [ ]:
processor.save_pretrained("./path_to_saved_model")

#Pushing your model to HF hub
The training results can now be uploaded to the Hub. To do so, execute the `push_to_hub` command and save the preprocessor object we created:

When we are done, We can submit our checkpoint to the [`hf-speech-bench`](https://huggingface.co/spaces/huggingface/hf-speech-bench) on push by setting the appropriate key-word arguments (kwargs):

In [ ]:
kwargs = {
    "dataset_tags": "tobiolatunji/afrispeech-200",
    "dataset": "AfriSpeech",  # a 'pretty' name for the training dataset
    "dataset_args": "config: en, split: test",
    "language": "en",
    "model_name": "Whisper Small En - Owos",  # a 'pretty' name for our model
    "finetuned_from": "openai/whisper-small",
    "tasks": "automatic-speech-recognition",
    "tags": "hf-asr-leaderboard",
}

In [ ]:
#trainer.push_to_hub(**kwargs)

#Predictions



In [ ]:
sample = next(iter(dataset_cln['validation']))['audio']
device = "cuda" if torch.cuda.is_available() else "cpu"
input_features = processor(next(iter(dataset_cln['validation']))['audio']["array"], sampling_rate=next(iter(dataset_cln['validation']))['audio']["sampling_rate"], return_tensors="pt").input_features.to(device) 
input_features.to(device)
model.to(device)

# generate token ids
predicted_ids = model.generate(input_features)

# decode token ids to text
transcription = processor.batch_decode(predicted_ids, skip_special_tokens=False)

transcription

In [ ]:
transcription = processor.batch_decode(predicted_ids, skip_special_tokens=True)
transcription

#### Loading the model from your local PC

In [ ]:
from transformers import WhisperProcessor, WhisperForConditionalGeneration
from datasets import load_dataset

# load model and processor
processor = WhisperProcessor.from_pretrained(training_args.output_dir)
model = WhisperForConditionalGeneration.from_pretrained(training_args.output_dir)
model.to(device)

In [ ]:
# generate token ids

predicted_ids = model.generate(input_features)
#decode token ids to text
transcription = processor.batch_decode(predicted_ids, skip_special_tokens=False)

transcription

In [ ]:
transcription = processor.batch_decode(predicted_ids, skip_special_tokens=True)
transcription

#Making Final Submission with 10 samples
* Load the data, 
* Load the processor,
* Load the model and predict


##### 

In [ ]:
def make_preds(batch):
    
    sample = batch['audio']
    device = "cuda" if torch.cuda.is_available() else "cpu"
    input_features = processor(sample["array"], sampling_rate=sample["sampling_rate"], return_tensors="pt").input_features.to(device) 
    input_features.to(device)

    # generate token ids
    predicted_ids = model.generate(input_features)
    

    # decode token ids to text
    transcription = processor.batch_decode(predicted_ids, skip_special_tokens=True)

    batch['preds'] = transcription[0]
    
#     return [batch['audio_id'], batch['preds']]
    return batch

In [ ]:
result = dataset_cln['validation'].map(make_preds)

In [ ]:
import gc

del dataset_cln, eval_dataset, train_dataset
gc.collect()

In [ ]:
result = result.remove_columns(['audio', 'transcript', 'input_features', 'labels'])

In [ ]:
display(next(iter(result)))

In [ ]:
def return_pred(batch):
    pred = pd.DataFrame(columns=['ID', 'transcript'])
    pred.loc['ID'] = batch['audio_id']
    pred['transcript'] = batch['preds']
    return pred

In [ ]:
from joblib import Parallel, delayed

df = Parallel(n_jobs=-1, verbose=1, prefer="threads")(delayed(return_pred)(batch) for batch in result)
df = pd.concat(df)
df

In [ ]:
import pandas as pd

df['test_paths'] = df['ID'].apply(lambda x: x+'.wav')
df = df.drop(['ID'], axis=1)
df= df.merge(dev_meta, on=['ID', 'test_paths'], how='outer').merge(test_meta, on=['ID', 'test_paths'], how='outer')
df = df[['ID', 'transcript']]
df.to_csv('submission.csv', index=False)

df